In [1]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageDraw

def loadMesh(filename: str) -> list[float]:
    v = []
    vt = []
    vn = []

    vertices = []
    normal=[]
    texture=[]
    with open(filename, "r") as file:

        line = file.readline()

        while line:

            words = line.split(" ")
            if words[0]=="v":
                v.append(read_vertex_data(words))
            elif words[0]=="vt":
                vt.append(read_texture_data(words))
            elif words[0]=="vn":
                vn.append(read_normal_data(words))
            
            elif words[0]=="f":
                face = []
                for word in words[1:]:
                    w = word.split('/')

                    face.append(int(w[0]))
                    vertices.append(v[int(w[0])-1])
                    texture.append(vt[int(w[1])-1])
                    normal.append(vn[int(w[2])-1])


                # read_face_data(words, v, vertices)
            
            line = file.readline()

    return vertices,texture,normal
 


def read_vertex_data(words: list[str]) -> list[float]:
    #вершины
    return [
        float(words[1]),
        float(words[2]),
        float(words[3])
    ]
def read_texture_data(words: list[str]) -> list[float]:
    #вершины
    return [
        float(words[2]),
        float(words[3]),
        float(words[4]),
    ]
def read_normal_data(words: list[str]) -> list[float]:
    #вершины
    return [
        float(words[2]),
        float(words[3]),
        float(words[4])
    ]

    
def odnor(coord):
    new=[]
    for i in range(len(coord)):
        new.append(list(coord[i]))
        new[i].append(1)
    return new
img = Image.new("RGB",(2400,2400), (255,255,255))
points=(loadMesh('african_head.obj'))[0]
draw = ImageDraw.Draw(img)
new_c= points
for i in range(0,int(len(new_c))-2,3):
   
    draw.polygon([(600*(2-new_c[i][0]),600*(2-new_c[i][1])),(600*(2-new_c[i+1][0]),600*(2-new_c[i+1][1])),(600*(2-new_c[i+2][0]),600*(2-new_c[i+2][1]))], outline='black')
    
    
img.show()

Роберстсон

In [3]:
from numpy.linalg import solve
def robertson(coord,cam):
    back=[]
    coord=odnor(coord)
    # print(coord)
    telo=[]
    for i in range(0,len(coord)-2,3):
        a=[coord[i],coord[i+1],coord[i+2],[0,0,0,-1]]
        b=[-1,-1,-1,-1]
        c=solve(a,b)
        x_test=(coord[i][0]+coord[i+1][0]+coord[i+2][0])/3
        y_test=(coord[i][1]+coord[i+1][1]+coord[i+2][1])/3
        z_test=(coord[i][2]+coord[i+1][2]+coord[i+2][2])/3
        m=np.sign(x_test*c[0]+y_test*c[1]+c[2]*z_test+1)
        c=np.dot(m,c)
        # с=norm()
        # print(c)
        # telo.append(c)
    # print(telo)
    # new_t=np.matmul(telo,cam)
    # for i in range(0,len(new_t)-2):
    #     if new_t[i]>0:
            
        if np.dot(cam,c)>=0:
            back.append(coord[i][0:-1])
            back.append(coord[i+1][0:-1])
            back.append(coord[i+2][0:-1])
    return back

img = Image.new("RGB",(2400,2400), (255,255,255))
points=(loadMesh('african_head.obj'))[0]
norm=(loadMesh('african_head.obj'))[2]
draw = ImageDraw.Draw(img)
new_c= robertson(points,[0,0,1,0])
print(new_c)
for i in range(0,int(len(new_c)-2),3):  
     draw.polygon([(600*(2-new_c[i][0]),600*(2-new_c[i][1])),(600*(2-new_c[i+1][0]),600*(2-new_c[i+1][1])),(600*(2-new_c[i+2][0]),600*(2-new_c[i+2][1]))], outline='black')
img.show()     

[[0.134781, -0.14723, 0.48805], [0.131261, -0.132153, 0.49872], [0.14749, -0.135105, 0.489565], [0.134781, -0.14723, 0.48805], [0.14749, -0.135105, 0.489565], [0.153371, -0.140519, 0.477416], [0.075062, -0.0473306, 0.49955], [0.0695841, -0.0997942, 0.550277], [0.0302569, -0.0971868, 0.55389], [0.075062, -0.0473306, 0.49955], [0.0302569, -0.0971868, 0.55389], [0.000686924, -0.0534984, 0.505694], [0.0354892, -0.12785, 0.535159], [0.0389268, -0.109557, 0.550558], [0.0734835, -0.11538, 0.538029], [0.0354892, -0.12785, 0.535159], [0.0734835, -0.11538, 0.538029], [0.0745124, -0.135953, 0.523215], [0.107415, -0.144295, 0.5047], [0.0745124, -0.135953, 0.523215], [0.0734835, -0.11538, 0.538029], [0.107415, -0.144295, 0.5047], [0.0734835, -0.11538, 0.538029], [0.106386, -0.125623, 0.51513], [0.107415, -0.144295, 0.5047], [0.106386, -0.125623, 0.51513], [0.131261, -0.132153, 0.49872], [0.107415, -0.144295, 0.5047], [0.131261, -0.132153, 0.49872], [0.134781, -0.14723, 0.48805], [0.156224, -0.12370

Этот Робертсон

In [4]:
from numpy.linalg import solve
def robertson(coord,norm,cam):
    back=[]
    coord=odnor(coord)
    norm=odnor(norm)
    # print(coord)
    
    
    for i in range(0,len(norm)-2,3):
        nx=(norm[i][0]+norm[i+1][0]+norm[i+2][0]+1)/4
        ny=(norm[i][1]+norm[i+1][1]+norm[i+2][1]+1)/4
        nz=(norm[i][2]+norm[i+1][2]+norm[i+2][2]+1)/4
        n=[nx,ny,nz,1]
        new_t=np.dot(cam,n)
        if new_t>0:
            
        # if np.dot(cam,c)>=0:
            back.append(coord[i][0:-1])
            back.append(coord[i+1][0:-1])
            back.append(coord[i+2][0:-1])
        # print(len(back))
    return back

img = Image.new("RGB",(2400,2400), (255,255,255))
points=(loadMesh('african_head.obj'))[0]
draw = ImageDraw.Draw(img)
norm=(loadMesh('african_head.obj'))[2]
new_c= robertson(points,norm,[0,0,1,0])
print(new_c)
for i in range(0,int(len(new_c)-2),3):  
     draw.polygon([(600*(2-new_c[i][0]),600*(2-new_c[i][1])),(600*(2-new_c[i+1][0]),600*(2-new_c[i+1][1])),(600*(2-new_c[i+2][0]),600*(2-new_c[i+2][1]))], outline='black')
img.show()     

[[0.156224, -0.123705, 0.486735], [0.15792, -0.107798, 0.4677], [0.163784, -0.126996, 0.470144], [0.156224, -0.123705, 0.486735], [0.163784, -0.126996, 0.470144], [0.15968, -0.130861, 0.484794], [0.160668, -0.135649, 0.476189], [0.153371, -0.140519, 0.477416], [0.14749, -0.135105, 0.489565], [0.160668, -0.135649, 0.476189], [0.14749, -0.135105, 0.489565], [0.154687, -0.134655, 0.486373], [0.163784, -0.126996, 0.470144], [0.160668, -0.135649, 0.476189], [0.154687, -0.134655, 0.486373], [0.163784, -0.126996, 0.470144], [0.154687, -0.134655, 0.486373], [0.15968, -0.130861, 0.484794], [0.156224, -0.123705, 0.486735], [0.131606, -0.0726738, 0.476148], [0.15792, -0.107798, 0.4677], [0.221602, -0.335752, 0.330174], [0.275434, -0.301318, 0.24751], [0.307699, -0.239869, 0.292356], [0.221602, -0.335752, 0.330174], [0.307699, -0.239869, 0.292356], [0.250944, -0.275583, 0.380381], [0.319444, 0.00732526, 0.451091], [0.28667, -0.025805, 0.481549], [0.297006, -0.0758717, 0.454242], [0.319444, 0.00732

Z-буффер

In [6]:
from PIL import Image, ImageDraw
import numpy as np

def z_buffer(points,texture,normal, width, height):

    z_buffer = np.full((height, width), np.inf)
    img = Image.new('RGB', (width, height), 'black')
    draw = ImageDraw.Draw(img)
    
    for i in range(0,int(len(points)-2),3):
        triangle=[points[i],points[i+1],points[i+2]]
        nx=(normal[i][0]+normal[i+1][0]+normal[i+2][0]+1)/4
        ny=(normal[i][1]+normal[i+1][1]+normal[i+2][1]+1)/4
        nz=(normal[i][2]+normal[i+1][2]+normal[i+2][2]+1)/4
        n=[nx,ny,nz]

        draw_triangle(triangle,n, draw, z_buffer, width, height)
    
    return img

def draw_triangle(triangle,texture, draw, z_buffer, width, height):

    v0=[0,0,0]
    v1=[0,0,0]
    v2=[0,0,0]

    for i in range(3):
        v0[i], v1[i], v2[i] = 700*(1-triangle[0][i]),700*(1-triangle[1][i]),700*(1-triangle[2][i])
  
    min_x = max(0, min(v0[0], v1[0], v2[0]))
    max_x = min(width - 1, max(v0[0], v1[0], v2[0]))
    min_y = max(0, min(v0[1], v1[1], v2[1]))
    max_y = min(height - 1, max(v0[1], v1[1], v2[1]))
    camera_v = [0, 0,1]
    intensity = np.dot(camera_v, texture)

    if intensity < 0:
        return
    color = int(intensity * 255)
  
    for x in range(int(min_x), int(max_x) + 1):
        for y in range(int(min_y), int(max_y) + 1):
            if is_inside_triangle(x, y, v0, v1, v2):
                z = get_z_value(x, y, v0, v1, v2)
                if z < z_buffer[y, x]:
                    z_buffer[y, x] = z
                    draw.point((x, y), fill=color)

def is_inside_triangle(px, py, v0, v1, v2):

    def sign(p1, p2, p3):
        return (p1[0] - p3[0]) * (p2[1] - p3[1]) - (p2[0] - p3[0]) * (p1[1] - p3[1])

    b1 = sign((px, py), v0, v1) < 0.0
    b2 = sign((px, py), v1, v2) < 0.0
    b3 = sign((px, py), v2, v0) < 0.0

    return ((b1 == b2) and (b2 == b3))

def get_z_value(px, py, v0, v1, v2):
  
    def edge_function(v0, v1, p):
        return (p[0] - v0[0]) * (v1[1] - v0[1]) - (p[1] - v0[1]) * (v1[0] - v0[0])

    area = edge_function(v0, v1, v2)
    w0 = edge_function(v1, v2, (px, py)) / area
    w1 = edge_function(v2, v0, (px, py)) / area
    w2 = edge_function(v0, v1, (px, py)) / area

    return w0 * v0[2] + w1 * v1[2] + w2 * v2[2]


image = Image.new("RGB",(1200,1200), (255,255,255))
points=(loadMesh('african_head.obj'))[0]
texture=(loadMesh('african_head.obj'))[1]
width, height = 1400, 1400
image = z_buffer(points,texture, norm,width, height)
image.show()
